In [18]:
import sys
# setting path
sys.path.append('../')

import pandas as pd

In [19]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
RESULTS_PATH = MAIN_PATH + "results/metrics/"
hyperparam_name = "lambda_val"
hyperparam_show_name = "lambda"

In [20]:
def highlight_columns(df, cols_idx = None, mode="min"):
    if cols_idx is None:
        cols = df.columns
    else:
        cols = df.columns[cols_idx]
    new_df = df.copy()
    for i in range(len(cols)):
        new_col = ['{:5f}'.format(a) for a in df[f"{cols[i]}"]]
        # Bold max value in each column
        if mode == "max":
            bold_idx = df[f"{cols[i]}"].argmax()
        elif mode == "min":
            bold_idx = df[f"{cols[i]}"].argmin()
        new_col[bold_idx] = "\\mathbf{" + new_col[bold_idx] + "}"
        new_col = ["$" + s + "$" for s in new_col]
        new_df = new_df.assign(**{cols[i]: new_col})
    return new_df

# COMPARATIVA GLOBAL

In [21]:
path = RESULTS_PATH + "results_camelyon.csv"
df_cam = pd.read_csv(path)
df_cam = df_cam.drop(columns=["patch_size", "n_samples", "sigmaRui_sq"]+[c for c in df_cam.columns if "loss" in c])
path = RESULTS_PATH + "results_wssb.csv"
df_wssb = pd.read_csv(path)
df_wssb= df_wssb.drop(columns=["patch_size", "n_samples", "sigmaRui_sq"]+[c for c in df_wssb.columns if ("loss" in c) or ("rec" in c)])
new_df_wssb = df_wssb[["pretraining_epochs", hyperparam_name]]
new_df_wssb["wssb_mean_test_psnr_gt_h"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_h" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_psnr_gt_e"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_e" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_psnr_gt"] = (new_df_wssb[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]]).mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt_h"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_h" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt_e"] = df_wssb[[c for c in df_wssb.columns if "ssim_gt_e" in c]].mean(axis=1)
new_df_wssb["wssb_mean_test_ssim_gt"] = (new_df_wssb[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]]).mean(axis=1)

<ipython-input-21-6e6b77021bcb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_wssb["wssb_mean_test_psnr_gt_h"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_h" in c]].mean(axis=1)
<ipython-input-21-6e6b77021bcb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_wssb["wssb_mean_test_psnr_gt_e"] = df_wssb[[c for c in df_wssb.columns if "psnr_gt_e" in c]].mean(axis=1)
<ipython-input-21-6e6b77021bcb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [22]:
df = pd.merge(df_cam, new_df_wssb, on=[hyperparam_name,"pretraining_epochs"])
df = df.drop(columns=[c for c in df.columns if ("gt_h" in c) or ("gt_e" in c)])
df[hyperparam_name] = df[hyperparam_name].astype(str)
df = highlight_columns(df, cols_idx=[3], mode="min")
df = highlight_columns(df, cols_idx=[4, 5, 6, 7], mode="max")
df.columns = ["Mnet", "Pre. epochs", f"$\\{hyperparam_show_name}$", "Rec. MSE (Camelyon)", "Rec. PSNR (Camelyon)", "Rec. SSIM (Camelyon)", "GT PSNR (WSSB)", "GT SSIM (WSSB)"]
df = df.sort_values(by=["Pre. epochs", f"$\\{hyperparam_show_name}$"])
df.set_index(["Pre. epochs", f"$\\{hyperparam_show_name}$"], inplace=True)
df

Mnet  Rec. MSE (Camelyon)  Rec. PSNR (Camelyon)  \
Pre. epochs $\lambda$                                                          
0           0.001      resnet18in  $\mathbf{0.000073}$  $\mathbf{33.840837}$   
            0.01       resnet18in           $0.000668$           $22.980550$   
            0.1        resnet18in           $0.001344$           $18.886120$   

                      Rec. SSIM (Camelyon)        GT PSNR (WSSB)  \
Pre. epochs $\lambda$                                              
0           0.001      $\mathbf{0.991915}$  $\mathbf{23.553930}$   
            0.01                $0.976709$           $20.547534$   
            0.1                 $0.954720$           $20.170609$   

                            GT SSIM (WSSB)  
Pre. epochs $\lambda$                       
0           0.001      $\mathbf{0.963408}$  
            0.01                $0.923570$  
            0.1                 $0.921264$

In [23]:
styler = df.style
#styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Summary of results.", label="tab:results_summary",
    )
print(s)

\begin{table}
\centering
\caption{Summary of results.}
\label{tab:results_summary}
\begin{tabular}{cccccccc}
\toprule
 &  & Mnet & Rec. MSE (Camelyon) & Rec. PSNR (Camelyon) & Rec. SSIM (Camelyon) & GT PSNR (WSSB) & GT SSIM (WSSB) \\
Pre. epochs & $\lambda$ &  &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{0} & 0.001 & resnet18in & $\mathbf{0.000073}$ & $\mathbf{33.840837}$ & $\mathbf{0.991915}$ & $\mathbf{23.553930}$ & $\mathbf{0.963408}$ \\
 & 0.01 & resnet18in & $0.000668$ & $22.980550$ & $0.976709$ & $20.547534$ & $0.923570$ \\
 & 0.1 & resnet18in & $0.001344$ & $18.886120$ & $0.954720$ & $20.170609$ & $0.921264$ \\
\cline{1-8}
\bottomrule
\end{tabular}
\end{table}



## RECONSTRUCTION CAMELYON

In [27]:
path = RESULTS_PATH + "results_camelyon.csv"
df = pd.read_csv(path)
df = df.drop(columns=["patch_size", "n_samples", "sigmaRui_sq"] + ["camelyon_test_loss", "camelyon_test_loss_mse", "camelyon_test_loss_kl"])
df[hyperparam_name] = df[hyperparam_name].astype(str)
df = highlight_columns(df, cols_idx=[3], mode="min")
df = highlight_columns(df, cols_idx=[4, 5], mode="max")
#df["Version"] = ["New" for i in range(len(df))]
#sh_row = {"Version" : "Old", "patch_size" : 64, "pretraining_epochs" : 1, "n_samples" : "?", "lambda_val" : "$\\theta=0.8$", "sigmaRui_sq" : "0.05", "camelyon_test_mse_rec" : "0.0712", "camelyon_test_ssim_rec" : "?", "camelyon_test_ssim_rec" : "?"}
#df = df.append(sh_row, ignore_index=True)
#df.columns = ["Patch size", "Pretraining epochs", "Num. samples", "$\theta$", "$\sigma_{Rui}^2$", "Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM", "Version"]
#df = df[["Version"] + list(df.columns[:-1])]
#df.columns = ["Patch size", "Pretraining epochs", "Num. samples", "$\\theta$", "$\sigma_{Rui}^2$", "Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM"]
df.columns = ["Mnet", "Pre. epochs", f"$\\{hyperparam_show_name}$", "Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM"]
df = df.sort_values(by=["Pre. epochs", f"$\\{hyperparam_show_name}$"])
df.set_index(["Mnet", "Pre. epochs", f"$\\{hyperparam_show_name}$"], inplace=True)
df

Reconstruction MSE   Reconstruction PSNR  \
Mnet       Pre. epochs $\lambda$                                              
resnet18in 0           0.001      $\mathbf{0.000073}$  $\mathbf{33.840837}$   
                       0.01                $0.000668$           $22.980550$   
                       0.1                 $0.001344$           $18.886120$   

                                  Reconstruction SSIM  
Mnet       Pre. epochs $\lambda$                       
resnet18in 0           0.001      $\mathbf{0.991915}$  
                       0.01                $0.976709$  
                       0.1                 $0.954720$

In [ ]:
styler = df.style
#styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Results on Camelyon test dataset.", label="tab:results_camelyon",
    )
print(s)
# Fix header

## RECONSTRUCTION WSSB

In [34]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns= ["patch_size", "n_samples", "sigmaRui_sq"] + [c for c in df.columns if "loss" in c])
df[hyperparam_name] = df[hyperparam_name].astype(str)
df = df[["mnet_name", "pretraining_epochs", hyperparam_name] + [c for c in df.columns if "mse_rec" in c]]
df["wssb_mean_test_mse_rec"] = df.mean(axis=1)
df = highlight_columns(df, cols_idx=[3,4,5,6], mode="min")
df.columns = ["Mnet", "Pre. epochs", f"$\\{hyperparam_show_name}$", "Lung Rec. MSE", "Breast Rec. MSE", "Colon Rec. MSE", "Mean Rec. MSE"]
df = df.sort_values(by=["Pre. epochs", f"$\\{hyperparam_show_name}$"])
df.set_index(["Mnet", "Pre. epochs", f"$\\{hyperparam_show_name}$"], inplace=True)
df

<ipython-input-34-dfc73d3fa13a>:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df["wssb_mean_test_mse_rec"] = df.mean(axis=1)


Lung Rec. MSE      Breast Rec. MSE  \
Mnet       Pre. epochs $\lambda$                                             
resnet18in 0           0.001      $\mathbf{0.000067}$  $\mathbf{0.000137}$   
                       0.01                $0.000139$           $0.000441$   
                       0.1                 $0.000178$           $0.000935$   

                                       Colon Rec. MSE        Mean Rec. MSE  
Mnet       Pre. epochs $\lambda$                                            
resnet18in 0           0.001      $\mathbf{0.000095}$  $\mathbf{0.000075}$  
                       0.01                $0.000207$           $0.000197$  
                       0.1                 $0.000245$           $0.000339$

In [ ]:
styler = df.style
#styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Reconstruction MSE on WSSB dataset.", label="tab:mse_rec_wssb",
    )
print(s)

# TABLAS DEL MEJOR MODELO

## Per-stain per-organ metrics

In [36]:
HYPERPARAM_VAL = 0.001
PRETRAINING_EPOCHS = 0

In [37]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
#theta_list = list(df["theta_val"])
#pe_list = list(df["pretraining_epochs"])
df = df.drop(columns=[c for c in df.columns if "loss" in c])
df = df[df["pretraining_epochs"] == PRETRAINING_EPOCHS] 
df = df[df[hyperparam_name] == HYPERPARAM_VAL]
df = df[[c for c in df.columns if "psnr_gt" in c]]
df["Name"] = [f"BCDnet p.e.={PRETRAINING_EPOCHS}, $\\theta={HYPERPARAM_VAL}$"]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", 
    "wssb_Lung_test_psnr_gt_h" : 27.06, "wssb_Lung_test_psnr_gt_e" : 25.69, 
    "wssb_Breast_test_psnr_gt_h" : 21.81, "wssb_Breast_test_psnr_gt_e" : 24.53, 
    "wssb_Colon_test_psnr_gt_h" : 20.32, "wssb_Colon_test_psnr_gt_e" : 21.70}, ignore_index=True)
df = df.append({"Name" : "BKSVD", 
    "wssb_Lung_test_psnr_gt_h" : 32.67, "wssb_Lung_test_psnr_gt_e" : 30.61, 
    "wssb_Breast_test_psnr_gt_h" : 32.20, "wssb_Breast_test_psnr_gt_e" : 29.43, 
    "wssb_Colon_test_psnr_gt_h" : 34.08, "wssb_Colon_test_psnr_gt_e" : 33.32}, ignore_index=True)

df["wssb_mean_test_psnr_gt_h"] = df[[c for c in df.columns if "psnr_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt_e"] = df[[c for c in df.columns if "psnr_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt"] = df[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]].mean(axis=1)
df = highlight_columns(df, cols_idx=[1,2,3,4,5,6,7,8,9], mode="max")
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

<ipython-input-37-2f9d4e98aa5a>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BCDnet (old)",
<ipython-input-37-2f9d4e98aa5a>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BKSVD",


BCDnet p.e.=0, $\theta=0.001$ BCDnet (old)                 BKSVD
Organ  Stain                                                                 
Lung   H                       $27.763270$  $27.060000$  $\mathbf{32.670000}$
       E                       $24.957645$  $25.690000$  $\mathbf{30.610000}$
Breast H                       $24.639019$  $21.810000$  $\mathbf{32.200000}$
       E                       $20.752524$  $24.530000$  $\mathbf{29.430000}$
Colon  H                       $23.597372$  $20.320000$  $\mathbf{34.080000}$
       E                       $19.613751$  $21.700000$  $\mathbf{33.320000}$
Mean   H                       $25.333220$  $23.063333$  $\mathbf{32.983333}$
       E                       $21.774640$  $23.973333$  $\mathbf{31.120000}$
       Mean                    $23.553930$  $23.518333$  $\mathbf{32.051667}$

In [40]:
styler = df.style
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Ground truth PSNR on WSSB dataset.", label="tab:gt_psnr_wssb",
    )
print(s)

\begin{table}
\centering
\caption{Ground truth PSNR on WSSB dataset.}
\label{tab:gt_psnr_wssb}
\begin{tabular}{ccccccccccccccccccccccccccccccccccc}
\toprule
 & mnet_name & patch_size & pretraining_epochs & n_samples & lambda_val & sigmaRui_sq & wssb_Lung_test_mse_rec & wssb_Lung_test_psnr_rec & wssb_Lung_test_ssim_rec & wssb_Lung_test_mse_gt_h & wssb_Lung_test_mse_gt_e & wssb_Lung_test_psnr_gt_h & wssb_Lung_test_psnr_gt_e & wssb_Lung_test_ssim_gt_h & wssb_Lung_test_ssim_gt_e & wssb_Breast_test_mse_rec & wssb_Breast_test_psnr_rec & wssb_Breast_test_ssim_rec & wssb_Breast_test_mse_gt_h & wssb_Breast_test_mse_gt_e & wssb_Breast_test_psnr_gt_h & wssb_Breast_test_psnr_gt_e & wssb_Breast_test_ssim_gt_h & wssb_Breast_test_ssim_gt_e & wssb_Colon_test_mse_rec & wssb_Colon_test_psnr_rec & wssb_Colon_test_ssim_rec & wssb_Colon_test_mse_gt_h & wssb_Colon_test_mse_gt_e & wssb_Colon_test_psnr_gt_h & wssb_Colon_test_psnr_gt_e & wssb_Colon_test_ssim_gt_h & wssb_Colon_test_ssim_gt_e \\
\midrule
0 & res

In [41]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
df = df[df["pretraining_epochs"] == PRETRAINING_EPOCHS] 
df = df[df[hyperparam_name] == HYPERPARAM_VAL]
#df = df[df["pretraining_epochs"] == 0] 
df = df[[c for c in df.columns if "ssim_gt" in c]]
df["Name"] = [f"BCDnet p.e.={PRETRAINING_EPOCHS}, $\\theta={HYPERPARAM_VAL}$"]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", 
    "wssb_Lung_test_ssim_gt_h" : 0.9518, "wssb_Lung_test_ssim_gt_e" : 0.8201, 
    "wssb_Breast_test_ssim_gt_h" : 0.8962, "wssb_Breast_test_ssim_gt_e" : 0.7321, 
    "wssb_Colon_test_ssim_gt_h" : 0.8969, "wssb_Colon_test_ssim_gt_e" : 0.8560}, ignore_index=True)
df = df.append({"Name" : "BKSVD", 
    "wssb_Lung_test_ssim_gt_h" : 0.9764, "wssb_Lung_test_ssim_gt_e" : 0.9461, 
    "wssb_Breast_test_ssim_gt_h" : 0.9801, "wssb_Breast_test_ssim_gt_e" : 0.9632, 
    "wssb_Colon_test_ssim_gt_h" : 0.9826, "wssb_Colon_test_ssim_gt_e" : 0.9646}, ignore_index=True)
df["wssb_mean_test_ssim_gt_h"] = df[[c for c in df.columns if "ssim_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt_e"] = df[[c for c in df.columns if "ssim_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt"] = df[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]].mean(axis=1)
df = highlight_columns(df, cols_idx=[1,2,3,4,5,6,7,8,9], mode="max")
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

<ipython-input-41-e9418c9aa95a>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BCDnet (old)",
<ipython-input-41-e9418c9aa95a>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BKSVD",


BCDnet p.e.=0, $\theta=0.001$ BCDnet (old)                BKSVD
Organ  Stain                                                                
Lung   H               $\mathbf{0.976623}$   $0.951800$           $0.976400$
       E               $\mathbf{0.996242}$   $0.820100$           $0.946100$
Breast H                        $0.947401$   $0.896200$  $\mathbf{0.980100}$
       E               $\mathbf{0.978590}$   $0.732100$           $0.963200$
Colon  H                        $0.954419$   $0.896900$  $\mathbf{0.982600}$
       E                        $0.927173$   $0.856000$  $\mathbf{0.964600}$
Mean   H                        $0.959481$   $0.914967$  $\mathbf{0.979700}$
       E               $\mathbf{0.967335}$   $0.802733$           $0.957967$
       Mean                     $0.963408$   $0.858850$  $\mathbf{0.968833}$

In [42]:
styler = df.style
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Ground truth SSIM on WSSB dataset.", label="tab:gt_ssim_wssb",
    )
print(s)

\begin{table}
\centering
\caption{Ground truth SSIM on WSSB dataset.}
\label{tab:gt_ssim_wssb}
\begin{tabular}{ccccc}
\toprule
 &  & BCDnet p.e.=0, $\theta=0.001$ & BCDnet (old) & BKSVD \\
Organ & Stain &  &  &  \\
\midrule
\multirow[c]{2}{*}{Lung} & H & $\mathbf{0.976623}$ & $0.951800$ & $0.976400$ \\
 & E & $\mathbf{0.996242}$ & $0.820100$ & $0.946100$ \\
\cline{1-5}
\multirow[c]{2}{*}{Breast} & H & $0.947401$ & $0.896200$ & $\mathbf{0.980100}$ \\
 & E & $\mathbf{0.978590}$ & $0.732100$ & $0.963200$ \\
\cline{1-5}
\multirow[c]{2}{*}{Colon} & H & $0.954419$ & $0.896900$ & $\mathbf{0.982600}$ \\
 & E & $0.927173$ & $0.856000$ & $\mathbf{0.964600}$ \\
\cline{1-5}
\multirow[c]{3}{*}{Mean} & H & $0.959481$ & $0.914967$ & $\mathbf{0.979700}$ \\
 & E & $\mathbf{0.967335}$ & $0.802733$ & $0.957967$ \\
 & Mean & $0.963408$ & $0.858850$ & $\mathbf{0.968833}$ \\
\cline{1-5}
\bottomrule
\end{tabular}
\end{table}



In [44]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
#df[[c for c in df.columns if "mse_rec" in c]]
df = df[df["pretraining_epochs"] == PRETRAINING_EPOCHS] 
df = df[df[hyperparam_name] == HYPERPARAM_VAL]
df = df[[c for c in df.columns if "mse_rec" in c]]
df["wssb_mean_test_mse_rec"] = df.mean(axis=1)
df["Name"] = [f"BCDnet p.e.={PRETRAINING_EPOCHS}, $\\theta={HYPERPARAM_VAL}$"]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", "wssb_Lung_test_mse_rec" : 0.0066, "wssb_Breast_test_mse_rec" : 0.0172, "wssb_Colon_test_mse_rec" : 0.0092, "wssb_mean_test_mse_rec" : 0.0110}, ignore_index=True)
df = df.append({"Name" : "BKSVD", "wssb_Lung_test_mse_rec" : 0.0004, "wssb_Breast_test_mse_rec" : 0.0003, "wssb_Colon_test_mse_rec" : 0.0003, "wssb_mean_test_mse_rec" : 0.0003}, ignore_index=True)
df = highlight_columns(df, cols_idx=[1,2,3,4], mode="min")
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Breast", "Colon", "Mean"]
df = df[["Organ"] + list(df.columns[:-1])]
df

<ipython-input-44-295ac2b7b1b0>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BCDnet (old)", "wssb_Lung_test_mse_rec" : 0.0066, "wssb_Breast_test_mse_rec" : 0.0172, "wssb_Colon_test_mse_rec" : 0.0092, "wssb_mean_test_mse_rec" : 0.0110}, ignore_index=True)
<ipython-input-44-295ac2b7b1b0>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name" : "BKSVD", "wssb_Lung_test_mse_rec" : 0.0004, "wssb_Breast_test_mse_rec" : 0.0003, "wssb_Colon_test_mse_rec" : 0.0003, "wssb_mean_test_mse_rec" : 0.0003}, ignore_index=True)


,Organ,"BCDnet p.e.=0, $\theta=0.001$",BCDnet (old),BKSVD
0,Lung,$\mathbf{0.000067}$,$0.006600$,$0.000400$
1,Breast,$\mathbf{0.000137}$,$0.017200$,$0.000300$
2,Colon,$\mathbf{0.000095}$,$0.009200$,$0.000300$
3,Mean,$\mathbf{0.000100}$,$0.011000$,$0.000300$


In [45]:
styler = df.style
styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns))]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Reconstruction MSE on WSSB dataset.", label="tab:mse_rec_wssb",
    )
print(s)

\begin{table}
\centering
\caption{Reconstruction MSE on WSSB dataset.}
\label{tab:mse_rec_wssb}
\begin{tabular}{cccc}
\toprule
Organ & BCDnet p.e.=0, $\theta=0.001$ & BCDnet (old) & BKSVD \\
\midrule
Lung & $\mathbf{0.000067}$ & $0.006600$ & $0.000400$ \\
Breast & $\mathbf{0.000137}$ & $0.017200$ & $0.000300$ \\
Colon & $\mathbf{0.000095}$ & $0.009200$ & $0.000300$ \\
Mean & $\mathbf{0.000100}$ & $0.011000$ & $0.000300$ \\
\bottomrule
\end{tabular}
\end{table}

